<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import pandas as pd
import tweepy

keys = pd.read_csv("D:/KL/NLP/20210513/Llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [12]:
tweets = tweepy.Cursor(api.search,
                       q=['paro nacional -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

t = [tweet for tweet in tweets]

t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
        
    t_processed.append(temp)

In [15]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date']
data.head()

,tweets,date
0,"Defensores de Derechos Humanos, NOS ESTÁN MATA...",2021-05-20 22:24:29
1,¿Para esto NO hay prensa internacional? ¿No ha...,2021-05-20 22:24:15
2,Debido a la situación de orden público por cue...,2021-05-20 22:23:59
3,Lo bueno es que los de Nacional también están ...,2021-05-20 22:23:47
4,@ClaudiaLopez Tu enfermedad no justifica tu ac...,2021-05-20 22:22:48


In [16]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [17]:
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,texto_limpio,links,hashtags,mentions
0,"Defensores de Derechos Humanos, NOS ESTÁN MATA...",2021-05-20 22:24:29,"Defensores de Derechos Humanos, NOS ESTÁN MATA...",[],[],[]
1,¿Para esto NO hay prensa internacional? ¿No ha...,2021-05-20 22:24:15,¿Para esto NO hay prensa internacional? ¿No ha...,[https://t.co/LrSOsQvCcJ],[],"[@CIDH, @CorteIDH_SOI, @MisionONUCol, @ONUHuma..."
2,Debido a la situación de orden público por cue...,2021-05-20 22:23:59,Debido a la situación de orden público por cue...,[https://t.co/GsoEyW2twm],[],[]
3,Lo bueno es que los de Nacional también están ...,2021-05-20 22:23:47,Lo bueno es que los de Nacional también están ...,[],[],[]
4,@ClaudiaLopez Tu enfermedad no justifica tu ac...,2021-05-20 22:22:48,Tu enfermedad no justifica tu actuación en es...,[],[],[@ClaudiaLopez]
...,...,...,...,...,...,...
95,"Poema del Paro Nacional de 2019, Bogotá, Carre...",2021-05-20 20:34:34,"Poema del Paro Nacional de 2019, Bogotá, Carre...",[],[],[]
96,@jflafaurie @Fedegan Jajaaja le importa tanto ...,2021-05-20 20:33:23,Jajaaja le importa tanto ayudar al gremio qu...,[],[],"[@jflafaurie, @Fedegan]"
97,El Paro Nacional continúa: habrá movilizacione...,2021-05-20 20:32:28,El Paro Nacional continúa: habrá movilizacione...,[https://t.co/hac49Czkez],[],[]
98,@alejo7valencia @lcvelez @lafm Para que tanta ...,2021-05-20 20:31:39,Para que tanta vaina mejor seguir así sin n...,[],[],"[@alejo7valencia, @lcvelez, @lafm]"


In [23]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [24]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,texto_limpio,links,hashtags,mentions,hora,emojis
0,"Defensores de Derechos Humanos, NOS ESTÁN MATA...",2021-05-20 22:24:29,"Defensores de Derechos Humanos, NOS ESTÁN MATA...",[],[],[],22:24:00,[🙏]
1,¿Para esto NO hay prensa internacional? ¿No ha...,2021-05-20 22:24:15,¿Para esto NO hay prensa internacional? ¿No ha...,[https://t.co/LrSOsQvCcJ],[],"[@CIDH, @CorteIDH_SOI, @MisionONUCol, @ONUHuma...",22:24:00,[]
2,Debido a la situación de orden público por cue...,2021-05-20 22:23:59,Debido a la situación de orden público por cue...,[https://t.co/GsoEyW2twm],[],[],22:23:00,[]
3,Lo bueno es que los de Nacional también están ...,2021-05-20 22:23:47,Lo bueno es que los de Nacional también están ...,[],[],[],22:23:00,[]
4,@ClaudiaLopez Tu enfermedad no justifica tu ac...,2021-05-20 22:22:48,Tu enfermedad no justifica tu actuación en es...,[],[],[@ClaudiaLopez],22:22:00,[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [25]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
11,20:31:00,2
39,20:32:00,1
54,20:33:00,1
35,20:34:00,1
45,20:35:00,1
...,...,...
20,22:13:00,2
17,22:16:00,2
47,22:22:00,1
10,22:23:00,2


## Serie de tiempo

In [26]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [27]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

In [28]:
layout = go.Layout(title="Número de tuits por hora")

In [29]:
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

## Barras

In [42]:
from collections import Counter
tempEmojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values, [])).keys(),Counter(sum(data.emojis.values, [])).values())))
tempEmojis.columns = ['emoji', 'cnt']
tempEmojis.sort_values('cnt', ascending=False, inplace=True)

In [45]:
trace = go.Bar(x=tempEmojis.emoji.values,
               y=tempEmojis.cnt.values)

layout = go.Layout(title='Emojis mas usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)